# Filter 3: Filtering Marconi100 jobs that share a node

## Reading data (output of `2023_05_22_marconi_jobs.ipynb`)

In [1]:
import pandas as pd

df_jobs_single = pd.read_csv("../Marconi-test/plugin=job_table/metric=job_info_marconi100/a_0_f12_singlenode.csv")
df_jobs_multi = pd.read_csv("../Marconi-test/plugin=job_table/metric=job_info_marconi100/a_0_f12_multinode.csv")

#print(df_jobs_multi.columns)

/tmp/ipykernel_24260/4178677611.py:3: DtypeWarning: Columns (62,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df_jobs_single = pd.read_csv("../Marconi-test/plugin=job_table/metric=job_info_marconi100/a_0_f12_singlenode.csv")


In [2]:
df_total_power = pd.read_parquet("../Marconi-test/plugin=ipmi_pub/metric=total_power/a_0.parquet")
df_total_power['node'] = pd.to_numeric(df_total_power['node'])
df_total_power['value'] = pd.to_numeric(df_total_power['value'])
print(df_total_power.columns)

Index(['timestamp', 'value', 'node'], dtype='object')


## Filtering for single node jobs

The idea is to add a new column to `df_total_power` with a list of jobs that run on each node at each timestamp. Then we filter jobs that appear sharing with another job

In [4]:
import ast
import numpy as np

#df_jobs_f1_single = df_jobs_f1[df_jobs_f1["num_nodes"]==1]

#removing malformed "nodes"
#df_jobs_f1_single = df_jobs_f1_single[df_jobs_f1_single["nodes"].str.match("\[\d+\]")].copy().reset_index(drop=True)
df_jobs_single["node"] = [ast.literal_eval(x)[0] for x in df_jobs_single['nodes']]


#use this for smaller inputs
TEST_SAMPLE_SIZE = 20000
sample = df_jobs_single[0:TEST_SAMPLE_SIZE].copy().reset_index(drop=True)

#sample = df_jobs_single

#df_total_power['job_ids'] = ""
group = df_total_power.groupby(by="node")

#for debugging
#result = [group.get_group(x[0])["timestamp"].between(x[1], x[2]).value_counts() for x in sample[["node", "start_time", "end_time"]].values.tolist()]

#i know that x[0] is node, x[1] is start_time, and x[2] is end_time
#each element of the list is a subseries of the total_power of a certain mode
#with "1" at the timestamps where the "job_id" ran
result = [[group.get_group(x[0])["timestamp"].between(x[1], x[2]).astype(int).replace(0, np.nan).dropna().astype(str), x[3]] for x in sample[["node", "start_time", "end_time", "job_id"]].values.tolist()]
#print(result)


if True:
    #print(type(group.get_group(512)))
    #group.get_group(x[0])

    #result[0] = Series with timestamps with True where the job run, and False otherwise
    #result[1] = job id
    #result.values.to_list()

    # now i replace the "1"s with the "job_id"
    #each element of the list is a subseries of the total_power of a certain mode
    #with the "job_id" value at the timestamps where the "job_id" ran
    result2 = [x[0].replace("1.0", str(x[1])) for x in result]
    #print(result2)

    #finally i concatenate each of the series by index
    # while joining the values where indices overlap
    # i hope the indices here are consistent with the indices in df_total_power 
    #concatenated_series = pd.concat(result2).groupby(level=0).apply(','.join).replace(to_replace=r'^(0,)*0$', value="0", regex=True)
    concatenated_series = pd.concat(result2).groupby(level=0).apply(','.join).rename("job_ids")
    #print(concatenated_series)

    # use the below code for the full run
    #joined_total_power = df_total_power.merge(concatenated_series, left_index=True, right_index=True, how="left")

    #use the below code for a rest run
    joined_total_power = df_total_power.merge(concatenated_series, left_index=True, right_index=True, how="inner")
    print(joined_total_power)

    #result3 = pd.concat(result2, verify_integrity=True)
    #print(result3[result3.values != 0])

    #sample["has_profile"] = result
    #df_jobs_f123_single = sample[sample["has_profile"]==True]

    #perc_jobs_filtered = (len(df_jobs_f123_single)/len(df_jobs_single))*100
    #print(str(perc_jobs_filtered), "% of the original job table")

                    timestamp  value  node                  job_ids
9102652   2022-05-09 22:00:00    820   616                  5217003
9102653   2022-05-09 22:00:20    900   616                  5217003
9102654   2022-05-09 22:00:40    880   616                  5217003
9102655   2022-05-09 22:01:00    780   616                  5217003
9102656   2022-05-09 22:01:20    680   616                  5217003
...                       ...    ...   ...                      ...
129200697 2022-05-28 21:58:20   1060   910  2891088,1851572,2540764
129200698 2022-05-28 21:58:40   1120   910  2891088,1851572,2540764
129200699 2022-05-28 21:59:00   1160   910  2891088,1851572,2540764
129200700 2022-05-28 21:59:20   1200   910  2891088,1851572,2540764
129200701 2022-05-28 21:59:40   1140   910  2891088,1851572,2540764

[9249948 rows x 4 columns]


## Saving the test data

In [5]:
#joined_total_power.to_csv("../Marconi-test/plugin=ipmi_pub/metric=total_power/power_jobids_test.csv", index=False)

## Finding the shared job ids

In [21]:
#job_ids = pd.Series([[int(y) for y in x.split(",")] for x in joined_total_power["job_ids"].drop_duplicates().values.tolist()])

all_job_ids=[]
_ = pd.Series([[all_job_ids.append(y) for y in x.split(",")] for x in joined_total_power["job_ids"].drop_duplicates().values.tolist()])

all_job_ids = pd.Series(all_job_ids).drop_duplicates()

nodeshare_job_ids = []
_ = pd.Series([[nodeshare_job_ids.append(y) for y in x.split(",") if len(x.split(",")) > 1] for x in joined_total_power["job_ids"].drop_duplicates().values.tolist()])
nodeshare_job_ids = pd.Series(nodeshare_job_ids).drop_duplicates()

exclusive_job_ids = all_job_ids[~all_job_ids.isin(nodeshare_job_ids)]

print("Nodeshare Job ratio: ", len(nodeshare_job_ids)/len(all_job_ids))
exclusive_job_ids
#all_job_ids

Nodeshare Job ratio:  0.4406


0        5217003
9        3861114
10       3435667
11       6191535
12       4796760
          ...   
36834     920827
37131    5378422
37132    3264690
37135    3215425
37143    5427250
Length: 11188, dtype: object

### Getting the profile of the not nodeshared jobs

In [ ]:
#x[0] = timestamp, x[1] = value (total_power), x[2] = node, x[3] = job_ids

result = [x for x in joined_total_power.values if np.any(pd.Series(x[3].split(",")).isin(exclusive_job_ids).values) == True]

df_result = pd.DataFrame(result, columns=["timestamp", "value", "node", "job_id"])


#result
#joined_total_power.values
#joined_total_power.values[0,3]

In [49]:
df_exclusive_jobs = df_jobs_single[df_jobs_single["job_id"].isin(exclusive_job_ids.astype(int))]
df_exclusive_jobs
#exclusive_job_ids
#df_jobs_single = pd.read_csv("../Marconi-test/plugin=job_table/metric=job_info_marconi100/a_0_f12_singlenode.csv")



,accrue_time,alloc_node,alloc_sid,array_job_id,array_max_tasks,array_task_id,array_task_str,array_task_throttle,assoc_id,batch_flag,...,tres_per_job,tres_per_node,tres_per_socket,tres_per_task,tres_req_str,user_id,wait4switch,wckey,node,has_profile
0,NaN,NaN,NaN,3231593,NaN,464.0,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:1,NaN,NaN,NaN,1234,NaN,NaN,458,True
2,NaN,NaN,NaN,3915009,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,1188,NaN,NaN,291,True
7,NaN,NaN,NaN,1492464,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,1702,NaN,NaN,677,True
15,NaN,NaN,NaN,6098620,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,1702,NaN,NaN,363,True
24,NaN,NaN,NaN,5198558,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,1702,NaN,NaN,694,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993,NaN,NaN,NaN,4687208,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:0,NaN,NaN,NaN,1167,NaN,NaN,688,True
19994,NaN,NaN,NaN,2119760,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,161,NaN,NaN,441,True
19996,NaN,NaN,NaN,5661358,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,161,NaN,NaN,962,True
19997,NaN,NaN,NaN,2586253,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:1,NaN,NaN,NaN,112,NaN,NaN,367,True


### Saving test results

In [57]:
df_result.to_csv("../Marconi-test/plugin=ipmi_pub/metric=total_power/power_jobids_test_exclusive.csv", index=False)
df_exclusive_jobs.to_csv("../Marconi-test/plugin=job_table/metric=job_info_marconi100/a_0_f12_singlenode_test_exclusive.csv", index=False)

## Filtering for multinode jobs

In [7]:
import ast
import numpy as np

#df_jobs_f1_single = df_jobs_f1[df_jobs_f1["num_nodes"]==1]

#removing malformed "nodes"
#df_jobs_f1_single = df_jobs_f1_single[df_jobs_f1_single["nodes"].str.match("\[\d+\]")].copy().reset_index(drop=True)
df_jobs_multi["node"] = [ast.literal_eval(x) for x in df_jobs_multi['nodes']]

#print(df_jobs_multi["node"])

if True:


    #use this for smaller inputs
    TEST_SAMPLE_SIZE = 1000
    sample = df_jobs_multi[0:TEST_SAMPLE_SIZE].copy().reset_index(drop=True)

    #sample = df_jobs_single

    #df_total_power['job_ids'] = ""
    group = df_total_power.groupby(by="node")

    #for debugging
    #result = [group.get_group(x[0])["timestamp"].between(x[1], x[2]).value_counts() for x in sample[["node", "start_time", "end_time"]].values.tolist()]

    #i know that x[0] is the node list, x[1] is start_time, and x[2] is end_time
    #each element of the list is a subseries of the total_power of a certain mode
    #with "1" at the timestamps where the "job_id" ran
    # i replace the "0"s (where the timestamp is not betweenx[1], x[2]) with nans just to remove these values with dropna.
    result = [[[group.get_group(y)["timestamp"].between(x[1], x[2]).astype(int).replace(0, np.nan).dropna().astype(str) for y in x[0]], x[3]] for x in sample[["node", "start_time", "end_time", "job_id"]].values.tolist()]
   
    #result = [[group.get_group(x[0])["timestamp"].between(x[1], x[2]).astype(int).replace(0, np.nan).dropna().astype(str), x[3]] for x in sample[["node", "start_time", "end_time", "job_id"]].values.tolist()]
    #print(result)


    if True:
        #print(type(group.get_group(512)))
        #group.get_group(x[0])

        #result[0] = Series with timestamps with True where the job run, and False otherwise
        #result[1] = job id
        #result.values.to_list()

        # now i replace the "1"s with the "job_id"
        #each element of the list is a subseries of the total_power of a certain mode
        #with the "job_id" value at the timestamps where the "job_id" ran
        result2 = [pd.concat([y.replace("1.0", str(x[1])) for y in x[0]]) for x in result]

        #result2 = [x[0].replace("1.0", str(x[1])) for x in result]
        #print(result2)
        if True:
            #finally i concatenate each of the series by index
            # while joining the values where indices overlap
            # i hope the indices here are consistent with the indices in df_total_power 
            #concatenated_series = pd.concat(result2).groupby(level=0).apply(','.join).replace(to_replace=r'^(0,)*0$', value="0", regex=True)
                        
            concatenated_series = pd.concat(result2).groupby(level=0).apply(','.join).rename("job_ids")
            #print(concatenated_series)

            # use the below code for the full run
            #joined_total_power = df_total_power.merge(concatenated_series, left_index=True, right_index=True, how="left")

            #use the below code for a rest run
            joined_total_power = df_total_power.merge(concatenated_series, left_index=True, right_index=True, how="inner")
            print(joined_total_power)

            #result3 = pd.concat(result2, verify_integrity=True)
            #print(result3[result3.values != 0])

            #sample["has_profile"] = result
            #df_jobs_f123_single = sample[sample["has_profile"]==True]

            #perc_jobs_filtered = (len(df_jobs_f123_single)/len(df_jobs_single))*100
            #print(str(perc_jobs_filtered), "% of the original job table")

                    timestamp  value  node  job_ids
18342767  2022-05-19 22:00:00    980   137   717969
18342768  2022-05-19 22:00:20    980   137   717969
18342769  2022-05-19 22:00:40   1000   137   717969
18342770  2022-05-19 22:01:00   1000   137   717969
18342771  2022-05-19 22:01:20   1020   137   717969
...                       ...    ...   ...      ...
129200199 2022-05-28 19:12:20    580   910  4254976
129200200 2022-05-28 19:12:40   1220   910  4254976
129200201 2022-05-28 19:13:00    820   910  4254976
129200202 2022-05-28 19:13:20    520   910  4254976
129200203 2022-05-28 19:13:40    420   910  4254976

[5082058 rows x 4 columns]


### Finding the shared job IDs

In [4]:
#job_ids = pd.Series([[int(y) for y in x.split(",")] for x in joined_total_power["job_ids"].drop_duplicates().values.tolist()])

all_job_ids=[]
_ = pd.Series([[all_job_ids.append(y) for y in x.split(",")] for x in joined_total_power["job_ids"].drop_duplicates().values.tolist()])

all_job_ids = pd.Series(all_job_ids).drop_duplicates()

nodeshare_job_ids = []
_ = pd.Series([[nodeshare_job_ids.append(y) for y in x.split(",") if len(x.split(",")) > 1] for x in joined_total_power["job_ids"].drop_duplicates().values.tolist()])
nodeshare_job_ids = pd.Series(nodeshare_job_ids).drop_duplicates()

exclusive_job_ids = all_job_ids[~all_job_ids.isin(nodeshare_job_ids)]

print("Nodeshare Job ratio: ", len(nodeshare_job_ids)/len(all_job_ids))
exclusive_job_ids
#all_job_ids

Nodeshare Job ratio:  0.018


0        717969
1       2142977
2       2572647
3       1906617
4       3865315
         ...   
1020    5290204
1021    4727631
1022     207237
1023    3724077
1024     595230
Length: 982, dtype: object

### Getting the profile of not nodeshared jobs 

In [5]:
#x[0] = timestamp, x[1] = value (total_power), x[2] = node, x[3] = job_ids

result = [x for x in joined_total_power.values if np.any(pd.Series(x[3].split(",")).isin(exclusive_job_ids).values) == True]

df_result = pd.DataFrame(result, columns=["timestamp", "value", "node", "job_id"])


#result
#joined_total_power.values
#joined_total_power.values[0,3]

In [8]:
df_exclusive_jobs = df_jobs_multi[df_jobs_multi["job_id"].isin(exclusive_job_ids.astype(int))]
df_exclusive_jobs

,accrue_time,alloc_node,alloc_sid,array_job_id,array_max_tasks,array_task_id,array_task_str,array_task_throttle,assoc_id,batch_flag,...,tres_per_job,tres_per_node,tres_per_socket,tres_per_task,tres_req_str,user_id,wait4switch,wckey,node,has_profile
0,NaN,NaN,NaN,2238969,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,1381,NaN,NaN,"[115, 150, 708, 710]",True
1,NaN,NaN,NaN,2086453,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,1380,NaN,NaN,"[464, 465, 466, 491, 492, 493, 509, 510, 512, ...",True
2,NaN,NaN,NaN,1803092,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,696,NaN,NaN,"[540, 541, 543, 544, 546, 547, 548, 551, 552, ...",True
3,NaN,NaN,NaN,956733,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,609,NaN,NaN,"[238, 773, 831, 924]",True
4,NaN,NaN,NaN,3032910,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,640,NaN,NaN,"[531, 532, 533, 535, 537, 538]",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,3176171,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,1430,NaN,NaN,"[36, 78, 237, 692]",True
996,NaN,NaN,NaN,3469816,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,1380,NaN,NaN,"[460, 461, 480, 481, 482, 503, 509, 511, 520, ...",True
997,NaN,NaN,NaN,2622845,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,826,NaN,NaN,"[40, 80, 161, 201, 240, 280, 360, 401, 441, 60...",True
998,NaN,NaN,NaN,2029327,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,gres:gpu:4,NaN,NaN,NaN,150,NaN,NaN,"[361, 405]",True


### Saving test results

In [9]:
df_result.to_csv("../Marconi-test/plugin=ipmi_pub/metric=total_power/power_jobids_test_exclusive_multinode.csv", index=False)
df_exclusive_jobs.to_csv("../Marconi-test/plugin=job_table/metric=job_info_marconi100/a_0_f12_singlenode_test_exclusive_multinode.csv", index=False)

## Plotting exclusive single node jobs

In [1]:
import pandas as pd

df_jobs_viz = pd.read_csv("../Marconi-test/plugin=job_table/metric=job_info_marconi100/a_0_f12_singlenode_test_exclusive.csv")

df_power = pd.read_csv("../Marconi-test/plugin=ipmi_pub/metric=total_power/power_jobids_test_exclusive.csv")
df_power['node'] = pd.to_numeric(df_power['node'])
df_power['value'] = pd.to_numeric(df_power['value'])

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
import ast

#gfig = None

def call_workflow(index, window_size):
    get_job_energy_profile(index, window_size)

def get_job_energy_profile(index, window_size):
    global current_job_id, current_hostname
    annot_str=''
    job=df_jobs_viz.iloc[index,:]
    nodes=ast.literal_eval(job['nodes'])
    start_time=job["start_time"]
    end_time=job["end_time"]
    for node in nodes:
        print(node)
        df_node = df_power.loc[df_power['node'] == node]
        df_node_job = df_node.loc[df_node['timestamp'].between(start_time, end_time)].sort_values(by="timestamp").reset_index(drop=True)
        df_node_job["rolling_mean"] = df_node_job.sort_values(by="timestamp").reset_index()["value"].rolling(window=window_size, center=True).mean()
        if len(df_node_job) > 0:
            #print(df_node_job.describe(), end="\r")
            plot_energy_profile(df_node_job, annot_str)
    print(nodes)
     
    #plot_energy_profile(job_energy_profile, annot_str)

    #current_job_id=str(job['job_id'])
    #current_hostname=energy_host
    ##
    #describe=job_energy_profile.describe(percentiles=[.10, .25, .5, .75, .90])
    #describe['job_id']=job['job_id']
    #describe['socket']=socket
    #describe['pp0']=describe[right_pp0]
    #describe['DRAM']=describe[right_DRAM]
    #describe['stat']=describe.index
    #describe=describe.reset_index(drop=True)[arr_cols]  
    #print(describe)  


def plot_energy_profile(profile, annot_str):
    TINY_SIZE = 2
    SMALL_SIZE = 5
    MEDIUM_SIZE = 25
    BIGGER_SIZE = 50
    FIG_WIDTH = 50
    FIG_HEIGHT = 20

    global gfig    

    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=BIGGER_SIZE)     # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    scatterplot_kwargs={'s': 50, 'palette': 'plasma'}
    lineplot_kwargs={'linewidth': 5}

    plt.clf()
    fig = plt.figure(figsize=(FIG_WIDTH,FIG_HEIGHT))
    #ax = sns.boxplot(x='stat', y='value', data=plot_data, showfliers=False, hue='reading',
    #             linewidth=TINY_SIZE)

        #print(profile_cpy["value"])
    ax = sns.scatterplot(data=profile, x='timestamp', y='rolling_mean', **scatterplot_kwargs)
    #ax = sns.lineplot(data=profile, x='timestamp', y='value', **lineplot_kwargs)

    ## SET BORDERS SIZE AND WIDTH
    [line.set_linewidth(TINY_SIZE) for line in ax.spines.values()]
    [line.set_markersize(TINY_SIZE) for line in ax.yaxis.get_ticklines()]
    [line.set_markeredgewidth(TINY_SIZE) for line in ax.yaxis.get_ticklines()]
    [line.set_markersize(SMALL_SIZE) for line in ax.xaxis.get_ticklines()]
    [line.set_markeredgewidth(TINY_SIZE) for line in ax.xaxis.get_ticklines()]
    #ax.text(x=0.1,y=0.5,
    #        s=annot_str,
    #        fontdict=dict(color='red',size=MEDIUM_SIZE),
    #        bbox=dict(facecolor='yellow',alpha=0.5),
    #        horizontalalignment='left',
    #        verticalalignment='center',
    #        transform=ax.transAxes)
    ax.set_ylabel('Processor Power (Watts)')
    ax.set_xlabel('Timestamp')
    ax.get_xaxis().set_visible(False)
    gfig = fig

button = widgets.Button(description="Save to PDF")
output = widgets.Output()

display(button, output)

## TODO: Pass on b the data (job_id, hostname, etc)
def on_button_clicked(b):
    with output:
        #fig=plt.gcf()
        #print(gfig)
        fig_filename='../Figures/marconi100_interact_plot_'+current_hostname+'_'+current_job_id+'.pdf'
        gfig.savefig(fig_filename, format='pdf', dpi=300, bbox_inches='tight')
        print('Plot saved as '+fig_filename)

button.on_click(on_button_clicked)

interact(call_workflow, index=widgets.IntSlider(min=0, max=len(df_jobs_viz)-1, step=1, value=0),
         window_size=widgets.IntSlider(min=1, max=30, step=1, value=1));

Button(description='Save to PDF', style=ButtonStyle())

Output()

interactive(children=(IntSlider(value=0, description='index', max=11187), IntSlider(value=1, description='wind…

## Plotting exclusive multinode jobs

In [ ]:
import pandas as pd

df_jobs_viz = pd.read_csv("../Marconi-test/plugin=job_table/metric=job_info_marconi100/a_0_f12_singlenode_test_exclusive_multinode.csv")

df_power = pd.read_csv("../Marconi-test/plugin=ipmi_pub/metric=total_power/power_jobids_test_exclusive_multinode.csv")
df_power['node'] = pd.to_numeric(df_power['node'])
df_power['value'] = pd.to_numeric(df_power['value'])